<div style="text-align: center"><h1 style="text-decoration: underline;">DSML Project</h1></div>



This is the official Notebook of the DSML Project from Marc Rennefort, Kilian Lipinsky, Timo Hagelberg, Jan Behrendt-Emden and Paul Severin. In order to create this Project we used the following dataset: https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips-2023-2024-/n26f-ihde/about_data
<h4>1. Description</h4>
The goal of this Project is to predict the tip for ride-hailing in Chicago based on the travelled time and distance as well as the money spent, the weather and the information if the customer matched to a shared trip or not. 


In [5]:
#Note all your imports here
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
import datetime


In [6]:
data = pd.read_csv('Data/Chicago_RideHailing_Data.csv') #habt geduld, dauert beim laden

C:\Users\kilia\AppData\Local\Temp\ipykernel_11668\1788640310.py:1: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Data/Chicago_RideHailing_Data.csv') #habt geduld, dauert beim laden


In [28]:
#erstmal grundlegendes Verständnis der Daten
data.head()
data.info()
data.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27423899 entries, 0 to 27423898
Data columns (total 24 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Trip ID                     object 
 1   Trip Start Timestamp        object 
 2   Trip End Timestamp          object 
 3   Trip Seconds                float64
 4   Trip Miles                  float64
 5   Percent Time Chicago        float64
 6   Percent Distance Chicago    float64
 7   Pickup Census Tract         float64
 8   Dropoff Census Tract        float64
 9   Pickup Community Area       float64
 10  Dropoff Community Area      float64
 11  Fare                        float64
 12  Tip                         float64
 13  Additional Charges          float64
 14  Trip Total                  float64
 15  Shared Trip Authorized      object 
 16  Shared Trip Match           object 
 17  Trips Pooled                float64
 18  Pickup Centroid Latitude    float64
 19  Pickup Centroid Lon

,Trip Seconds,Trip Miles,Percent Time Chicago,Percent Distance Chicago,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tip,Additional Charges,Trip Total,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Dropoff Centroid Latitude,Dropoff Centroid Longitude
count,2.742226e+07,2.742390e+07,2.741730e+07,2.741632e+07,1.694196e+07,1.689560e+07,2.501902e+07,2.493498e+07,2.735579e+07,2.735579e+07,2.735579e+07,2.735579e+07,2.742390e+07,2.510101e+07,2.510101e+07,2.500703e+07,2.500703e+07
mean,1.109868e+03,6.880468e+00,9.395580e-01,9.337823e-01,1.703140e+10,1.703141e+10,2.809590e+01,2.892227e+01,1.615844e+01,1.248352e+00,4.426983e+00,2.183377e+01,1.030692e+00,4.188638e+01,-8.767095e+01,4.188728e+01,-8.767396e+01
std,8.058195e+02,7.702479e+00,2.411365e-01,2.507243e-01,3.425574e+05,3.476064e+05,2.169261e+01,2.216091e+01,1.285458e+01,2.860655e+00,4.204643e+00,1.665526e+01,2.075346e-01,6.977729e-02,6.912047e-02,6.958577e-02,7.441440e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,4.165022e+01,-8.791362e+01,4.165022e+01,-8.791362e+01
25%,5.420000e+02,1.900000e+00,1.000000e+00,1.000000e+00,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,7.500000e+00,0.000000e+00,2.320000e+00,1.157000e+01,1.000000e+00,4.186342e+01,-8.768932e+01,4.186790e+01,-8.769334e+01
50%,8.910000e+02,4.000000e+00,1.000000e+00,1.000000e+00,1.703128e+10,1.703128e+10,2.500000e+01,2.800000e+01,1.250000e+01,0.000000e+00,3.390000e+00,1.623000e+01,1.000000e+00,4.189251e+01,-8.765401e+01,4.189251e+01,-8.765401e+01
75%,1.450000e+03,9.100000e+00,1.000000e+00,1.000000e+00,1.703183e+10,1.703183e+10,3.700000e+01,3.900000e+01,2.000000e+01,1.000000e+00,5.330000e+00,2.560000e+01,1.000000e+00,4.192933e+01,-8.763087e+01,4.193465e+01,-8.763087e+01
max,7.178700e+04,5.267000e+02,6.800000e+01,1.430000e+02,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,8.550000e+02,1.960000e+02,1.227220e+03,1.234720e+03,8.000000e+00,4.202122e+01,-8.752995e+01,4.202122e+01,-8.752995e+01


In [18]:
#Entfernen der Spalten, die wir nicht brauchen

data_firstColumnsCleaned = data.drop(columns = ['Percent Time Chicago', 'Percent Distance Chicago', 'Pickup Census Tract', 'Dropoff Census Tract',
                            'Pickup Community Area', 'Dropoff Community Area', 'Additional Charges', 'Trips Pooled', 
                            'Pickup Centroid Latitude', 'Pickup Centroid Location', 'Dropoff Centroid Latitude', 'Dropoff Centroid Longitude', 'Dropoff Centroid Location', 'Pickup Centroid Longitude'])
data_firstColumnsCleaned.head()


,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Fare,Tip,Trip Total,Shared Trip Authorized,Shared Trip Match
0,8255c0b48b877bda14fbc2f490b6e42f359c44ec,09/19/2023 06:15:00 PM,09/19/2023 07:15:00 PM,3109.0,23.5,32.5,0.0,40.70,False,False
1,8255c1529cdd828c3ce4e6bdc494c734e8432025,09/28/2023 12:15:00 PM,09/28/2023 12:45:00 PM,1640.0,14.7,30.0,0.0,38.74,False,False
2,8255c247359d6e148930ba860afa3282848c67ed,09/03/2023 05:15:00 PM,09/03/2023 05:30:00 PM,772.0,1.9,20.0,0.0,22.39,False,False
3,8255c37a57997b77f75da6af598a38fc28b4e832,09/13/2023 03:30:00 PM,09/13/2023 03:30:00 PM,452.0,1.1,10.0,0.0,14.07,False,False
4,8255c6b273a32b3a76d7dad03328abb229de7fbd,09/16/2023 01:00:00 PM,09/16/2023 01:00:00 PM,819.0,3.9,10.0,0.0,12.89,False,False


In [ ]:
#changing Format of the date
